# Colorspace - Mayavi Test - Volumetric Graphs

Create png files based on the screen dump from the atari800 emulation of colourspace.

Download our modified version of the atari800 emulator. This will allow us to create dump files of screen memory at specified trace points while colourspace is running.

In [3]:
import os
! rm -rf atari800
! git clone https://github.com/mwenge/atari800.git
os.chdir('atari800')
! git checkout memwatch
! ./autogen.sh
! ./configure --enable-monitormemorywatch
! make -j5
os.chdir('..')


Cloning into 'atari800'...
remote: Enumerating objects: 17549, done.
remote: Counting objects: 100% (1510/1510), done.
remote: Compressing objects: 100% (472/472), done./472)
remote: Total 17549 (delta 1069), reused 1422 (delta 1035), pack-reused 16039
Receiving objects: 100% (17549/17549), 5.75 MiB | 3.22 MiB/s, done.
Resolving deltas: 100% (13512/13512), done.
Branch 'memwatch' set up to track remote branch 'memwatch' from 'origin'.
Switched to a new branch 'memwatch'
configure.ac:204: installing './compile'
configure.ac:73: installing './install-sh'
configure.ac:73: installing './missing'
src/Makefile.am: installing './depcomp'

Now you need to run the configure script.  The configure script may take the
"--target=TARGET" option and various "--enable-FEATURE" and "--with-PACKAGE"
options.

Run "./configure --help" to see all available options.
Run "./configure --help=short" just to see the Atari800 FEATURE options.
Run "./configure --target=help" without a parameter to see the valid

config.status: executing depfiles commands
-------------------------------------------------------
                 CONFIGURATION RESULTS:
-------------------------------------------------------
Host OS...............................: linux
Target ...............................: default

Interface for video...................: sdl
Using screenshots?....................: yes
    Supported screenshot formats......: pcx png
Using cycle exact?....................: yes
Using the very slow computer support?.: no
Using the crash menu?.................: yes
Using the paged attribute array?......: no
Using per opcode cycles update?.......: no
Using the buffered log?...............: no
Using Altirra BIOS ROM?...............: yes
Using the monitor assembler?..........: yes
Using code breakpoints and history?...: yes
Using user-defined breakpoints?.......: no
Using monitor hints?..................: yes
Using 6502 opcode profiling?..........: no
Using TRACE monitor command?..........: no
Using rea

Run the atari800 emulator and write the screen memory dumps to a file.

In [4]:
import os
os.chdir('../psychedelia')
! make colourspace_nodemo.xex
os.chdir('../notebooks')
# 4327 is the point at which we've finished a round of painting
! ./atari800/src/atari800 -memwatchbpc 4327 -memwatchfile colorspace_mayavi_volumetric.txt ../psychedelia/bin/colourspace.xex

patch src/atari800/colourspace.asm -o src/atari800/colourspace_nodemo.asm < disable_demo.patch
patching file src/atari800/colourspace_nodemo.asm (read from src/atari800/colourspace.asm)
Hunk #1 succeeded at 4044 (offset 18 lines).
64tass -Wall -Wno-implied-reg --atari-xex -o bin/colourspace.xex -L bin/list-co1.txt -l bin/labels.txt src/atari800/colourspace_nodemo.asm
64tass Turbo Assembler Macro V1.56.2625?
64TASS comes with ABSOLUTELY NO WARRANTY; This is free software, and you
are welcome to redistribute it under certain conditions; See LICENSE!

Assembling file:   src/atari800/colourspace_nodemo.asm
Assembling file:   src/atari800/constants.asm
Assembling file:   src/atari800/patterns.asm
Assembling file:   src/atari800/presets.asm
Assembling file:   src/atari800/foreground.asm
Error messages:    None
Warning messages:  None
Passes:            3
Memory range:      $1f00-$7c66   $5d67
# the original xex file has an incorrect end-byte which we need to patch here.
dd if=bin/patch-atari

Read in the ram history from our dump file.

In [117]:
log_file = "colorspace_mayavi_volumetric.txt"
input_file = open(log_file,'r')

flatten = lambda l: [e for sublist in l for e in sublist]

ram_history = []
lines = input_file.readlines()
for i in range(1, len(lines), 131):
    # Screen RAM (7000-8000)
    raw_ram = [l[6:126].split() for l in lines[i:i+104]]
    pixel_ram = [int(v,16) for l in raw_ram for v in l][:0x1000]
    # Display list (8000-9000)
    raw_ram = [l[6:126].split() for l in lines[i+103:i+127]]
    display_list = [int(v,16) for l in raw_ram for v in l]
    # Color Registers
    color_registers = [int(i,16) for i in lines[i+129].split()[1:10]]
    ram_history += [(pixel_ram, color_registers, display_list)]

Helper functions for implementing the display list entries

In [118]:
def getByteForAddress(address,scr):
    if 0x7000 <= address < 0x8000:
        return scr[address - 0x7000]
    return 0

def drawBlankLines(pixels, byte, y, num, scr):
    #pixels[x, y] = ImageColor.getrgb('#000000')
    return num

def drawScanLine(pixels, byte, y, num, scr):
    for x in range(0,40):
        byte = getByteForAddress(address+x, scr)
        left_nibble = byte >> 4 & 0x0F 
        right_nibble = byte & 0x0F
        for n in range(0,4):
            pixels[y][(x*8)+n] = col_register[left_nibble]
        for n in range(0,4):
            pixels[y][(x*8)+n+4] = col_register[right_nibble]
    return 1

dlist_dict = {
    0x70: (drawBlankLines,1,1),
    0x90: (drawBlankLines,1,1),
    0x46: (drawBlankLines,3,1),
    0x4F: (drawScanLine,  3,1),
}
def display_list_entry(dl):
    # Consume entries in the display list one by one.
    i = 0
    while i < len(dl):
        b = dl[i]
        if b not in dlist_dict:
            yield (None,None,None,None,None)
        draw, advance, lines = dlist_dict[b]
        address = (dl[i+2]<<8 | dl[i+1]) if advance > 1 else 0
        i += advance
        yield (b, draw, address, lines,i)


Read in the ram  history and paint a png file using the display lists and the pixel data.

In [119]:
ROWS = 192
COLS = 320

pixel_history = []
#selected = [ram_history[x] for x in [188,236]]
for j, (scr, col_register, display_list) in enumerate(ram_history):
    pixels = [[0 for i in range(0,COLS)] for i in range(0,ROWS)]
    dlist = display_list_entry(display_list)
    x,y = 0,0
    while True:
        token, draw, address, num, i = next(dlist)
        if draw == None:
            break
        #print(hex(token),hex(address),num,i,display_list[i:i+2],len(display_list))
        y += draw(pixels, address, y, num, scr)
    pixel_history+= [pixels]

In [120]:
flatten = lambda l: [e for sublist in l for e in sublist]

pattern_breaks = []
for i,h in enumerate(pixel_history):
    num_pixels = sum(flatten(h))
    if num_pixels > 1:
        continue
    pattern_breaks += [i]

patterns = []
pp = 0
for p in pattern_breaks:
    patterns += [pixel_history[pp:p]]
    pp = p
len(patterns)

1

In [141]:
from PIL import Image, ImageColor, ImageChops
import colorspace_colors as cc
import numpy as np

def createVoxelData(screens):
    # prepare some coordinates
    width = len(screens[0][0])
    height = len (screens[0])
    x, y, z, colors = [],[],[],[]
    for zp,screen in enumerate(screens):
        for yp, row in enumerate(list(screen)):
            for xp, col in enumerate(row):
                # Skip black, white and gray cells
                if col in [0,1,0x0c]:
                    continue
                x+=[xp]
                y+=[yp]
                z+=[zp]
                colors+=[col]

    return (x,y,z,colors)


In [142]:
import numpy as np
from mayavi import mlab
mlab.init_notebook()

Notebook initialized with ipy backend.


In [176]:
col_register

[0, 24, 56, 88, 120, 152, 184, 216, 219]

In [192]:
screens = patterns[0]

(x,y,z,colors) = createVoxelData(screens)
mlab.close()
s = mlab.points3d(x, y, z, mode="cube")
s.glyph.scale_mode = 'scale_by_vector'
s.mlab_source.dataset.point_data.scalars = colors
# Retrieve the LUT of the surf object.
#lut = s.module_manager.scalar_lut_manager.lut.table.to_array()
#print(len(lut))
mlab.roll(125)
s

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

In [146]:
x = np.array([1,8])
y = np.array([1,8])
z = np.array([1,10])
value = np.array([0.2,0.3])
print(x,y,z,value)
mlab.close()
s = mlab.points3d(x, y, z, value,scale_mode="vector", mode="point")
s

[1 8] [1 8] [ 1 10] [0.2 0.3]


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…